# Week 2.6: Leakage-Controlled Evaluation

**Purpose**: Run inference on original vs sanitized text to measure actual F1 delta.

**Requires**: GPU runtime (T4 recommended)

**Note**: This notebook uses your existing Week 2 checkpoint (upload required).

In [ ]:
# 1. Clone Repository
!git clone https://github.com/AngadSingh22/Text2Diag.git
%cd Text2Diag

In [ ]:
# 2. Install Dependencies
!pip install -q torch transformers accelerate scikit-learn datasets pyyaml

In [ ]:
# 3. Download Raw Dataset
!python scripts/inspect_raw_datasets.py

In [ ]:
# 4. Build Canonical Dataset
!python scripts/02_build_reddit_canonical.py

In [ ]:
# 5. UPLOAD YOUR CHECKPOINT
# 
# Instructions:
# 1. On your local machine, ZIP the checkpoint folder:
#    - Go to: f:\Text2Diag\results_week2\results\week2\checkpoints\
#    - ZIP the 'checkpoint-4332' folder -> checkpoint-4332.zip
#
# 2. Run this cell - it will prompt you to upload the zip file
#
from google.colab import files
import zipfile
import os

print("Please upload checkpoint-4332.zip from your local machine...")
uploaded = files.upload()

# Extract the checkpoint
os.makedirs("results/week2/checkpoints", exist_ok=True)
for filename in uploaded.keys():
    print(f"Extracting {filename}...")
    with zipfile.ZipFile(filename, 'r') as zip_ref:
        zip_ref.extractall("results/week2/checkpoints/")
    
# Verify
!ls -la results/week2/checkpoints/
checkpoint_path = "results/week2/checkpoints/checkpoint-4332"
print(f"\nCheckpoint ready: {checkpoint_path}")

In [ ]:
# 6. Run Leakage-Controlled Evaluation
checkpoint_path = "results/week2/checkpoints/checkpoint-4332"

!python scripts/09_eval_sanitized.py \
    --checkpoint {checkpoint_path} \
    --data_dir data/processed/reddit_mh_windows \
    --out_dir results/week2/remediation \
    --sanitize_config configs/sanitize.yaml \
    --batch_size 32

In [ ]:
# 7. Check Results
!cat results/week2/remediation/leakage_eval_metrics.md

In [ ]:
# 8. View JSON Metrics
import json
with open('results/week2/remediation/leakage_eval_metrics.json', 'r') as f:
    metrics = json.load(f)
print(json.dumps(metrics, indent=2))

In [ ]:
# 9. Zip and Download Results
!zip -r w26_results.zip results/week2/remediation
from google.colab import files
files.download('w26_results.zip')